In [1]:
import os
import time
import numpy as np
import pandas as pd

from skimage import io

import matplotlib.pyplot as plt

from CellSeg.segmentation import Segmentation
%load_ext snakeviz

In [2]:
%pdb

Automatic pdb calling has been turned ON


In [3]:
path = "../data/ctrl/"
seg_filename = "ResultofC1-Composite_cp_masks-edited.tif"

pixel_size = dict(x_size = 0.183,
                  y_size = 0.183,
                  z_size = 0.5,)

In [4]:

image = io.imread(os.path.join(path, seg_filename))
seg = Segmentation(image, pixel_size, path)

In [5]:
%%snakeviz
start_time = time.time()
seg.perform_prerequisite()
# print("--- %s seconds ---" % (time.time() - start_time))

This folder npz is not empty, 
If you want to save file here, turn overright to True
 
*** Profile stats marshalled to file '/tmp/tmph96jlnk0'.
Embedding SnakeViz in this document...


In [6]:
%%snakeviz
start_time = time.time()
seg.cell_segmentation()
# print("--- %s seconds ---" % (time.time() - start_time))


 
*** Profile stats marshalled to file '/tmp/tmpxek4384l'.
Embedding SnakeViz in this document...


In [7]:
%%snakeviz
start_time = time.time()
seg.edge_segmentation()
# print("--- %s seconds ---" % (time.time() - start_time))
#2.62e+3s

 
*** Profile stats marshalled to file '/tmp/tmpaazyt44v'.
Embedding SnakeViz in this document...


In [5]:
%%snakeviz
start_time = time.time()
seg.face_segmentation()
# print("--- %s seconds ---" % (time.time() - start_time))
#631 sec
# 20 cells -> 207 sec avec np.where
# 51 sec without np.where
# 101 with sparse matrix

 
*** Profile stats marshalled to file '/tmp/tmpycw5uyvl'.
Embedding SnakeViz in this document...


In [6]:
import sparse
sp_mat = sparse.load_npz(os.path.join(path, "npz/" + str(int(102)) + ".npz"))
img_cell = sp_mat.todense()

In [8]:
sp_mat.coords.mean(axis=1)

array([141.18219088, 451.06476814, 201.99999204])

In [30]:
x

array([ 30,  30,  30, ..., 259, 259, 259])

In [5]:
from CellSeg.segmentation import find_ordered_neighbours, find_paths
edge_df = pd.read_csv(os.path.join(path, "edge_df.csv"))


In [23]:
sub_edges = edge_df[edge_df['id_im_1'] == 115]
sub_edges
# # cyclic path
# graph, ordered_neighbours, opp_cell = find_ordered_neighbours(sub_edges)

,Unnamed: 0,id_im_1,id_im_2,id_im_3,x_mean,y_mean,z_mean
10,10,115.0,102.0,158.0,174.042508,420.814028,151.891605
11,11,115.0,158.0,232.0,163.960912,403.350706,160.757872
12,12,115.0,391.0,232.0,170.923077,307.769231,271.923077
13,13,115.0,232.0,673.0,96.182622,372.628866,207.114875
14,14,115.0,719.0,102.0,144.212727,424.910909,197.385455


In [63]:
graph, ordered_neighbours1, opp_cell1 = find_paths(sub_edges)
graph, ordered_neighbours1, opp_cell1

(<networkx.classes.digraph.DiGraph at 0x7f05e9040e20>,
 [[None, 391.0, 232.0, 673.0, None],
  [None, 719.0, 102.0, 158.0, 232.0, 673.0, None]],
 {0: (391.0, None, 391.0, 391.0, 232.0),
  1: (232.0, 391.0, 232.0, 232.0, 673.0),
  2: (673.0, 232.0, 673.0, 673.0, None),
  3: (719.0, None, 719.0, 719.0, 102.0),
  4: (102.0, 719.0, 102.0, 102.0, 158.0),
  5: (158.0, 102.0, 158.0, 158.0, 232.0),
  6: (232.0, 158.0, 232.0, 232.0, 673.0),
  7: (673.0, 232.0, 673.0, 673.0, None)})

In [18]:
graph, ordered_neighbours2, opp_cell2 = find_ordered_neighbours(sub_edges)
graph, ordered_neighbours2, opp_cell2

(<networkx.classes.digraph.DiGraph at 0x7f05e9a0b250>,
 [array([[673., 115.],
         [115., 158.],
         [158., 382.],
         [382., 368.],
         [368., 673.]]),
  array([[673., 115.],
         [115., 158.],
         [158., 391.],
         [391., 382.],
         [382., 368.],
         [368., 673.]]),
  array([[673., 115.],
         [115., 391.],
         [391., 382.],
         [382., 368.],
         [368., 673.]])],
 {0: (115.0, 673.0, 115.0, 115.0, 158.0),
  1: (158.0, 115.0, 158.0, 158.0, 382.0),
  2: (382.0, 158.0, 382.0, 382.0, 368.0),
  3: (368.0, 382.0, 368.0, 368.0, 673.0),
  4: (673.0, 368.0, 673.0, 673.0, 115.0),
  5: (158.0, 115.0, 158.0, 158.0, 391.0),
  6: (391.0, 158.0, 391.0, 391.0, 382.0),
  7: (382.0, 391.0, 382.0, 382.0, 368.0),
  8: (115.0, 673.0, 115.0, 115.0, 391.0),
  9: (391.0, 115.0, 391.0, 391.0, 382.0)})

In [13]:
for c_op_index in opp_cell.keys():
    print(c_op_index)

0
1
2
3
4
5
6
7


In [55]:
k_max = np.max(list(opp_cell1.keys()))+1
for k, v in opp_cell2.items():
    opp_cell1[k+k_max] = v

In [59]:
def find_all_neighbours(sub_edges):
    graph, ordered_neighbours1, opp_cell1 = find_ordered_neighbours(sub_edges)
    graph, ordered_neighbours2, opp_cell2 = find_paths(sub_edges)

    k_max = len(opp_cell1.keys())
    for k, v in opp_cell2.items():
        opp_cell1[k + k_max] = v

    ordered_neighbours = ordered_neighbours1+ordered_neighbours2
    return ordered_neighbours, opp_cell1
find_all_neighbours(sub_edges)

([[None, 391.0, 232.0, 673.0, None],
  [None, 719.0, 102.0, 158.0, 232.0, 673.0, None]],
 {0: (391.0, None, 391.0, 391.0, 232.0),
  1: (232.0, 391.0, 232.0, 232.0, 673.0),
  2: (673.0, 232.0, 673.0, 673.0, None),
  3: (719.0, None, 719.0, 719.0, 102.0),
  4: (102.0, 719.0, 102.0, 102.0, 158.0),
  5: (158.0, 102.0, 158.0, 158.0, 232.0),
  6: (232.0, 158.0, 232.0, 232.0, 673.0),
  7: (673.0, 232.0, 673.0, 673.0, None)})